In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-11-07 03:34:57--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.01MB/s    in 0.9s    

2020-11-07 03:34:58 (1.01 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# 2.2 Load the data from Deliverable 1
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/sample_us.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [5]:
# 2.3 Recreate the vine_table DataFrame
# vine_df = df.select([])
vine_table = df.select(["review_id","star_rating","helpful_votes", "total_votes","vine", "verified_purchase"]).dropDuplicates(["review_id"])
# add drop duplicates to avoid errors when uploading to pgAdmin

vine_table.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R44NP0QG6E98W|          3|            1|          1|   N|                Y|
|R1Y4ZOUGFMJ327|          5|            0|          0|   N|                Y|
|R1W3QQZ8JKECCI|          5|            0|          0|   N|                Y|
| R47XBGQFP039N|          5|            0|          0|   N|                Y|
|R1YS3DS218NNMD|          5|            4|          4|   N|                N|
| RIDVQ4P3WJR42|          5|            1|          1|   N|                Y|
|R1JS8G26X4RM2G|          5|            0|          1|   N|                Y|
|R298788GS6I901|          5|            0|          0|   N|                Y|
|R3AHZWWOL0IAV0|          5|            0|          0|   N|                Y|
|R3OSJU70OIBWVE|          5|            0|          0|   N|     

In [31]:
votes20plus = vine_table.filter((vine_table['total_votes']) >= 20)
votes20plus.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [13]:
votes20plus_weight = votes20plus.filter(votes20plus["helpful_votes"]/votes20plus["total_votes"] >= 0.5)
votes20plus_weight.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [18]:
from pyspark.sql.functions import col, avg

paid_votes = votes20plus_weight.filter(votes20plus_weight['vine']== 'Y')
paid_votes.describe().show()

+-------+---------+-----------+-------------+-----------+----+-----------------+
|summary|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-------+---------+-----------+-------------+-----------+----+-----------------+
|  count|        0|          0|            0|          0|   0|                0|
|   mean|     null|       null|         null|       null|null|             null|
| stddev|     null|       null|         null|       null|null|             null|
|    min|     null|       null|         null|       null|null|             null|
|    max|     null|       null|         null|       null|null|             null|
+-------+---------+-----------+-------------+-----------+----+-----------------+



In [19]:
unpaid_votes = votes20plus_weight.filter(votes20plus_weight['vine']== 'N')
unpaid_votes.describe().show()

+-------+---------+-----------+-------------+-----------+----+-----------------+
|summary|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-------+---------+-----------+-------------+-----------+----+-----------------+
|  count|        0|          0|            0|          0|   0|                0|
|   mean|     null|       null|         null|       null|null|             null|
| stddev|     null|       null|         null|       null|null|             null|
|    min|     null|       null|         null|       null|null|             null|
|    max|     null|       null|         null|       null|null|             null|
+-------+---------+-----------+-------------+-----------+----+-----------------+



In [29]:
#  Determine the number of five-star reviews among Vine reviews
paid_five_star = paid_votes[paid_votes['star_rating']== 5].count()
paid_number = paid_votes.count()
print(paid_five_star)
print(paid_number)

0
0


In [30]:
#  Determine the percentage of five-star reviews among Vine reviews
percentage_five_star_vine = (paid_five_star) / (paid_number)

print(percentage_five_star_vine)

ZeroDivisionError: ignored